In [ ]:
#Standard Lib imports
import sys, os
from pathlib import Path
#from functools import partial
import getpass

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))


# Third party imports

# Local application imports
from sei import sei

#USR = getpass.getuser()
USR = 'rsilva'
print(f'Olá {USR}')
PWD = getpass.getpass("Digite sua Senha: ")

%load_ext autoreload
%autoreload 2

In [ ]:
s = sei.login_sei(USR, PWD, browser='Firefox', timeout=5, teste=False)

In [ ]:
s.mudar_lotação("GR01")

In [ ]:
s.mudar_lotação("GR01FI3")

In [ ]:
s.mudar_lotação("GR01AF")

In [ ]:
s.ver_detalhado()

In [ ]:
s.is_init_page()

In [ ]:
s.go_to_init_page()

In [ ]:
s.show_lat_menu()

In [ ]:
s.page.timeout = 2

In [ ]:
s.ver_detalhado()

In [ ]:
s.ver_todos()

In [ ]:
s.itera_processos()

In [ ]:
s.get_processos()

In [ ]:
s.go_to_blocos()

In [ ]:
num = "53504.009880/2019-93"
p = s.go_to_processo(num)

In [ ]:
p.is_open()

In [ ]:
p.incluir_documento('Externo')

In [ ]:
p.page.timeout = 2

In [ ]:
p.send_doc_por_email('0177520', ('ronaldokun@gmail.com', "Teste", "Medição RFeye"))

In [ ]:
p.abrir_pastas()

In [ ]:
p._get_acoes('0177509')

In [ ]:
p.is_open()